# SRE ReAct Agent

This SRE agent uses **LangGraph** to autonomously investigate Kubernetes cluster issues. It connects to a **Model Context Protocol (MCP) server** for Kubernetes access, allowing an **LLM (GPT-5 mini)** to use kubectl commands as tools. The agent follows a **ReAct pattern** (Reason + Act) - it analyzes the hotel reservation service, calls K8s tools when needed, and provides diagnostic reports without human intervention.

**Key components:**
- **MCP Client**: Provides kubectl tools to the LLM
- **LangGraph**: Orchestrates the investigation workflow  
- **ReAct Loop**: Agent → Tools → Agent until diagnosis complete

The agent automatically starts investigating when triggered by the system prompt.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI

#gpt5 = ChatOpenAI(model="gpt-5")
gpt5mini = ChatOpenAI(model="gpt-5-mini")
# gemini = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     google_api_key=os.getenv("GOOGLEAI_API_KEY") # Google AI Studio free api key
# )

## Testing MCP Client for Kubernetes

MCP K8s server: [mcp-server-kubernetes](https://github.com/Flux159/mcp-server-kubernetes)

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "kubernetes" : {
            "command": "npx",
            "args": ["mcp-server-kubernetes"],
            "transport": "stdio",
            "env": {
                "ALLOW_ONLY_NON_DESTRUCTIVE_TOOLS": "true"
            }
        }
    }
)

# NB: The cluster has to be running otherwise the tools won't be available
mcp_tools = await client.get_tools()

In [ ]:
#list all the tools
for tool in mcp_tools:
    print(f"🔧 {tool.name}: {tool.description}")

In [ ]:
# Tools allowed
tools_allowed = ["kubectl_get", "kubectl_describe", "kubectl_logs", "explain_resource", "list_api_resources", "ping"]

tools = []
for tool in mcp_tools:
    if tool.name in tools_allowed:
        tools.append(tool)


In [ ]:
for tool in tools:
    print(f"🔧 {tool.name}: {tool.description}")

## Baseline model

All the previous messages are passed to the LLM for determining the next action

This implementation requires a lot of tokens

In [ ]:
sytem_prompt = """
    You are an expert DevOps engineer who has been tasked with detecting anomalies in a deployed service.

    The service you are working with today is described below:
    {app_summary}

    You will use an MCP server which will provide you access to the kubernetes cluster.

    You will begin by analyzing the service's state and telemetry, and then submit your solution: describe the issue you have identified without fixing it!
    Also explain your reasoning and thought process behind the solution.
    Do not add any followup questions nor additional context.
    """

app_summary = """
    The application implements a hotel reservation service, build with Go and gRPC, and starting from the open-source project https://github.com/harlow/go-micro-services. The initial project is extended in several ways, including adding back-end in-memory and persistent databases, adding a recommender system for obtaining hotel recommendations, and adding the functionality to place a hotel reservation. 
"""

In [ ]:
llm_with_tools = gpt5mini.bind_tools(tools, parallel_tool_calls=False)

from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content=sytem_prompt.format(app_summary=app_summary))

# Node
async def sreAgent(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("sre-agent", sreAgent)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "sre-agent")
builder.add_conditional_edges(
    "sre-agent",
    # If the latest message (result) from sre-agent is a tool call -> tools_condition routes to tools
    # If the latest message (result) from sre-agent is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "sre-agent")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
print(react_graph.get_graph(xray=True).draw_mermaid())

In [ ]:
import time

async def test_async_graph(graph , human_message: str, trace_name: str = None):
    """Test the graph with proper async execution"""
    # Create a human message from the input
    messages = [HumanMessage(content=human_message)]
    
    # Start time tracking
    start_time = time.time()

    # Configuration for the graph execution
    config = {"recursion_limit": 50}
    if trace_name:
        config["run_name"] = trace_name
    
    # Invoke the graph asynchronously
    result = await graph.ainvoke({"messages": messages}, config)

    # Calculate execution time
    execution_time = time.time() - start_time
    
    # Print results
    for m in result['messages']:
        m.pretty_print()
    
    # Calculate token usage
    total_tokens = 0
    input_tokens = 0
    output_tokens = 0
    
    for m in result['messages']:
        if hasattr(m, 'usage_metadata'):
            if m.usage_metadata:
                if 'input_tokens' in m.usage_metadata:
                    input_tokens += m.usage_metadata['input_tokens']
                if 'output_tokens' in m.usage_metadata:
                    output_tokens += m.usage_metadata['output_tokens']
                if 'total_tokens' in m.usage_metadata:
                    total_tokens += m.usage_metadata['total_tokens']

    result["total_tokens"] = total_tokens
    result["input_tokens"] = input_tokens
    result["output_tokens"] = output_tokens
    result["execution_time"] = execution_time
    
    print(f"\nExecution time: {execution_time:.2f} seconds")
    print(f"Token usage: {total_tokens} total ({input_tokens} input, {output_tokens} output)")
    
    return result

In [ ]:
experiment_name = input("Enter experiment name: ")

if experiment_name.strip() == "":
    experiment_name = "SRE ReAct agent"

In [ ]:
human = ""

result_baseline = await test_async_graph(react_graph, human, trace_name=experiment_name)

In [ ]:
# Final AI response
result_baseline["messages"][-1].pretty_print()

In [ ]:
import os
import datetime
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def saveExperiment(experiment_name, result):
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    results_folder = "results"
    os.makedirs(results_folder, exist_ok=True)

    # Create the full path for the file
    file_path = os.path.join(results_folder, f"{timestamp}.txt")

    # Format and write messages to file
    with open(file_path, 'w') as f:
        f.write(f"{'='*50}\nEXPERIMENT SUMMARY\n{'='*50}\n\n")
        if experiment_name:
            f.write(f"Experiment Name: {experiment_name}\n")
        f.write(f"Total tokens used: {result['total_tokens']}\n")
        f.write(f"Input tokens: {result['input_tokens']}\n")
        f.write(f"Output tokens: {result['output_tokens']}\n")
        f.write(f"Execution time: {result['execution_time']:.2f} seconds\n\n")

        f.write(f"{'='*50}\nSYSTEM PROMPT\n{'='*50}\n\n")
        f.write(sys_msg.content + "\n\n")
        

        for i, message in enumerate(result['messages']):
            f.write(f"{'='*50}\nMESSAGE {i+1}\n{'='*50}\n")
            
            if isinstance(message, HumanMessage):
                f.write(f"Type: Human Message\n")
                f.write(f"Content: {message.content}\n\n")
            
            elif isinstance(message, AIMessage):
                f.write(f"Type: AI Message\n")
                if message.content:
                    f.write(f"Content: {message.content}\n\n")
                
                if hasattr(message, 'additional_kwargs') and 'tool_calls' in message.additional_kwargs:
                    tool_calls = message.additional_kwargs['tool_calls']
                    f.write(f"Tool Calls ({len(tool_calls)}):\n")
                    
                    for idx, tool_call in enumerate(tool_calls):
                        f.write(f"  Tool Call {idx+1}: {tool_call['id']}\n")
                        if 'function' in tool_call:
                            f.write(f"    Function: {tool_call['function']['name']}\n")
                            f.write(f"    Arguments: {tool_call['function']['arguments']}\n\n")
            
            elif isinstance(message, ToolMessage):
                f.write(f"Type: Tool Response\n")
                f.write(f"Tool Call ID: {message.tool_call_id}\n")
                f.write(f"Content:\n{message.content}\n\n")
            
            else:
                f.write(f"Type: Other Message Type ({type(message).__name__})\n")
                f.write(f"Content: {str(message)}\n\n")

    print(f"Messages exported to {file_path}")

In [ ]:
saveExperiment(experiment_name, result_baseline)

# Improve agent efficiency (in terms of tokens)

## Reduce context by passing only three messages

In [ ]:
MAX_MESSAGES = 7

In [ ]:
system_prompt_reduced_input = """
    You are an expert DevOps engineer who has been tasked with detecting anomalies in a deployed service.

    The service you are working with today is described below:
    {app_summary}

    Important constraint:
    You only have access to the {max_messages} most recent messages in this conversation. 
    Anything older is not visible to you, so do not reference or assume context beyond those last {max_messages} messages.

    Your task:
        - Begin by analyzing the service's state and telemetry.
        - Then submit your solution: describe the issue you have identified and the root cause (without fixing it).
        - Also explain your reasoning and thought process.
        - Do not add any followup questions nor additional context.
"""

app_summary = """
    The application implements a hotel reservation service, build with Go and gRPC, and starting from the open-source project https://github.com/harlow/go-micro-services. The initial project is extended in several ways, including adding back-end in-memory and persistent databases, adding a recommender system for obtaining hotel recommendations, and adding the functionality to place a hotel reservation. 
"""

In [ ]:
llm_with_tools = gpt5mini.bind_tools(tools, parallel_tool_calls=False)

from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AIMessage

# System message
sys_msg_reduced = SystemMessage(content=system_prompt_reduced_input.format(app_summary=app_summary, max_messages=MAX_MESSAGES))

# Node which limits the context window 
async def sreAgent_reducedInput(state: MessagesState):

   messages = state["messages"]

   if len(state["messages"]) <= MAX_MESSAGES:
      return {"messages": [llm_with_tools.invoke([sys_msg_reduced] + messages)]}
   
   # Reduce the context window by taking only the last few messages
   context_messages = []

   # Work backwards from the end
   i = len(messages) - 1
   while i >= 0 and len(context_messages) < (MAX_MESSAGES + 2):
      msg = messages[i]
      context_messages.insert(0, msg)
      
      # If this is a ToolMessage, make sure to include the AIMessage that called it
      if isinstance(msg, ToolMessage) and i > 0:
         prev_msg = messages[i-1]
         if isinstance(prev_msg, AIMessage) and hasattr(prev_msg, 'additional_kwargs'):
               if 'tool_calls' in prev_msg.additional_kwargs and prev_msg not in context_messages:
                  context_messages.insert(0, prev_msg)
      i -= 1

      return {"messages": [llm_with_tools.invoke([sys_msg_reduced] + context_messages)]}




In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("sre-agent", sreAgent_reducedInput)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "sre-agent")
builder.add_conditional_edges(
    "sre-agent",
    # If the latest message (result) from sre-agent is a tool call -> tools_condition routes to tools
    # If the latest message (result) from sre-agent is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "sre-agent")
react_graph_reduced = builder.compile()

# Show
display(Image(react_graph_reduced.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
experiment_name = input("Enter experiment name: ")

if experiment_name.strip() == "":
    experiment_name = "SRE ReAct agent"

In [ ]:
human = ""

result_reduced = await test_async_graph(react_graph_reduced, human, trace_name=experiment_name)

In [ ]:
# Final AI answer
result_reduced["messages"][-1].pretty_print()

In [ ]:
saveExperiment(experiment_name, result_reduced)

## Use custom structured schema

This approach improves token efficiency by replacing the full message history with a structured summary. Instead of passing all previous messages to the LLM, the agent maintains:

- **`insights`**: Key findings from each investigation step
- **`prev_steps`**: Concise descriptions of actions taken
- **`response`**: Final diagnosis when investigation is complete

After each tool execution, a `summarise` node extracts the most important information and updates the structured state. This allows the agent to maintain context while significantly reducing token usage compared to the baseline approach.

In [ ]:
from typing import TypedDict, List, Literal, Annotated
from langgraph.graph.message import add_messages, AnyMessage
import operator

class SREAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    app_summary: str
    insights: Annotated[list[str], operator.add]
    prev_steps: Annotated[list[str], operator.add]
    response: str
    final_output: str

In [ ]:
sre_agent_prompt = """
    You are an expert DevOps engineer who has been tasked with detecting anomalies in a deployed service.

    The service you are working with today is described below:
    {app_summary}

    You will use an MCP server which will provide you access to the Kubernetes cluster.

    Context:

    *Previous Steps:*
    {prev_steps}

    *Insights:*
    {insights}

    Your task:
        1. Begin by analyzing the service's state and telemetry using kubectl tools
        2. When you have identified the issue, call the submit_final_diagnosis tool with:
            - diagnosis: Describe the issue you have identified (without fixing it)
            - reasoning: Explain your reasoning and thought process behind the solution

    IMPORTANT: You must call submit_final_diagnosis when you're ready to conclude your investigation.
"""

In [ ]:
summarise_prompt = """
    You are an autonomous SRE agent for Kubernetes incident diagnosis.

    Context:

    Previous Insights: 
    {insights}
    
    Previous Steps:
    {prev_steps}

    Below are the latest two messages:
    {last_two_messages}

    Instructions:
    1. From the latest two messages, extract the most important new insight relevant for incident diagnosis or mitigation. Summarize it concisely.
    2. Write a concise description of only the most recent action taken including the tool used (not the whole list).  
"""

In [ ]:
from pydantic import BaseModel, Field

class UpdateAgentData(BaseModel):
    """
    Represents a step performed by the SRE agent.
    """
    insight: str = Field(..., description="Most important new finding")
    prev_step: str = Field(..., description="Concise description of the most recent action taken")

In [ ]:
llm_with_strct_output = gpt5mini.with_structured_output(UpdateAgentData)

In [ ]:
def get_insights_str(state):
    """Return a string with the formatted list of insights gathered during exploration"""
    if len(state["insights"]) > 0:
        return "\n- ".join([""] + state["insights"])
    else:
        return "No insights yet"
    
def get_prev_steps_str(state):
    """Return a string with the formatted list of previous steps performed during exploration"""
    if len(state["prev_steps"]) > 0:
        return "\n- ".join([""] + state["prev_steps"])
    else:
        return "No previous steps yet"

In [ ]:
from langchain_core.messages import HumanMessage

# Node used to summarise the infos given the two previous messages
async def summarise(state: SREAgentState):

    # Gather last two messages (tool call + tool response)
    last_messages = state["messages"][-2:]

    insights_str = get_insights_str(state)
    prev_step_str = get_prev_steps_str(state)

    prompt = HumanMessage(content=summarise_prompt.format(
        prev_steps = prev_step_str,
        insights=insights_str,
        last_two_messages=last_messages))

    data = llm_with_strct_output.invoke([prompt])

    return {"insights" : [data.insight], "prev_steps" : [data.prev_step]}

In [ ]:
from typing import Annotated
from langgraph.types import Command
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool, InjectedToolCallId

# Tool used to submit the final response
@tool
def submit_final_diagnosis(
    diagnosis: str, 
    reasoning: str,
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Submit the final diagnosis when investigation is complete.
    
    Args:
        diagnosis: The issue you have identified (without fixing it)
        reasoning: Your reasoning and thought process behind the diagnosis
    
    Returns:
        Command to update state and end workflow
    """
    final_response = f"Diagnosis:\n{diagnosis}\n\nReasoning:\n{reasoning}"
    
    return Command(
        update={
            "response": final_response, # Add in the final graph state the final answer
            "messages": [
                ToolMessage(
                    content="Final diagnosis submitted successfully. Investigation complete.",
                    tool_call_id=tool_call_id
                )
            ]
        },
        goto="format-output" # End the loop cycle
    )

# Append the tool for submission to the list of tools (MCP servers)
completion_tool = submit_final_diagnosis
tools_with_completion = tools + [completion_tool]

In [ ]:
async def sreAgent(state: SREAgentState):

    insights_str = get_insights_str(state)
    prev_step_str = get_prev_steps_str(state)

    prompt = HumanMessage(content=sre_agent_prompt.format(
        prev_steps=prev_step_str, 
        insights=insights_str, 
        app_summary=state["app_summary"]
    ))

    # Use tools with completion (for the submission)
    llm_with_completion_tools = gpt5mini.bind_tools(tools_with_completion, parallel_tool_calls=False)
    return {"messages": [llm_with_completion_tools.invoke([prompt])]}

In [ ]:
async def format_response(state: SREAgentState):

    insights_str = get_insights_str(state)
    prev_step_str = get_prev_steps_str(state)

    message = "**Results of the analysis**\n\n"

    message += "Steps performed:\n"
    message += prev_step_str

    message += "\n\nInsights gathered:\n"
    message += insights_str

    message += "\n\nFinal report\n"

    message += state["response"]

    return {"final_output" : message}

In [ ]:
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

# Build the graph
builder = StateGraph(SREAgentState)

# Add nodes
builder.add_node("sre-agent", sreAgent)
builder.add_node("tools", ToolNode(tools_with_completion)) # Tool node is executing the tool called in the previous message
builder.add_node("summarise", summarise) # Node to reduce the raw data into a schema
builder.add_node("format-output", format_response)

# Add edges
builder.add_edge(START, "sre-agent")

# Conditional edge from sre-agent
builder.add_conditional_edges(
    "sre-agent",
    #Use in the conditional_edge to route to the ToolNode if the last message has tool calls. Otherwise, route to the end.
    tools_condition,
)

# After tools, decide whether to summarise or end
def after_tools_condition(state: SREAgentState):
    # If response is filled, investigation is complete (end of the workflow)
    if state.get("response"):
        return "format-output"
    return "summarise"

builder.add_conditional_edges(
    "tools",
    after_tools_condition,
    {
        "summarise": "summarise",
        "format-output": "format-output"
    }
)

# After summarise, continue investigation (go to sre-agent)
builder.add_edge("summarise", "sre-agent")
builder.add_edge("format-output", END)

# Compile the graph
structured_graph = builder.compile()

# Show the graph
display(Image(structured_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
print(structured_graph.get_graph(xray=True).draw_mermaid())

In [ ]:
import time

async def test_structured_graph(graph, app_summary:str, human_message: str = "", trace_name: str = None):
    """Test the structured graph with SREAgentState"""
    # Create initial state with SREAgentState structure
    initial_state = {
        "messages": [HumanMessage(content=human_message)] if human_message else [],
        "insights": [],
        "prev_steps": [],
        "response": "",
        "final_output" : "",
        "app_summary" : app_summary
    }
    
    # Start time tracking
    start_time = time.time()

    # Configuration for the graph execution
    config = {"recursion_limit": 50}
    if trace_name:
        config["run_name"] = trace_name
    
    # Invoke the graph asynchronously
    result = await graph.ainvoke(initial_state, config)

    # Calculate execution time
    execution_time = time.time() - start_time

    return result

In [ ]:
experiment_name = input("Enter experiment name: ")

if experiment_name.strip() == "":
    experiment_name = "SRE structured format agent"

In [ ]:
app_summary = """
    The application implements a hotel reservation service, build with Go and gRPC, and starting from the open-source project https://github.com/harlow/go-micro-services. The initial project is extended in several ways, including adding back-end in-memory and persistent databases, adding a recommender system for obtaining hotel recommendations, and adding the functionality to place a hotel reservation. 
"""

In [ ]:
human = ""
structured_result = await test_structured_graph(structured_graph, app_summary, human, experiment_name)

In [ ]:
print(structured_result["final_output"])

In [ ]:
for step in structured_result["prev_steps"]:
    print(step)

In [ ]:
for insight in structured_result["insights"]:
    print(insight)

In [ ]:
print(structured_result["response"])

In [ ]:
for message in structured_result["messages"]:
    message.pretty_print()